# **Case Datarisk - Score de Crédito**

## **Importa as bibliotecas e Carrega os Dados**

In [12]:
import sys
import os

# Pega o diretório de trabalho atual (que é /notebooks)
notebook_dir = os.getcwd()
# Sobe um nível para o diretório raiz do projeto
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Adiciona a raiz do projeto ao sys.path se ainda não estiver lá
if project_root not in sys.path:
    print(f"Adicionando a raiz do projeto ao path: {project_root}")
    sys.path.append(project_root)

TODO: CatBoostEncoder

In [13]:
from src.RankCountVectorizer import RankCountVectorizer
from src.load_df import load_df
from src.convert_to_datetime import convert_to_datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticker

import requests
from pathlib import Path

from sklearn.metrics import roc_curve

In [14]:
# Carregar os DataFrames
df_targets = load_df("../data/targets.parquet")

df_cadastral = load_df("../data/base_cadastral.parquet")
df_emprestimos = load_df("../data/historico_emprestimos.parquet")
df_submissao = load_df("../data/base_submissao.parquet")
df_parcelas = load_df("../data/historico_parcelas.parquet")
dicionario = load_df("../data/dicionario_dados.csv")

In [15]:
df_targets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71043 entries, 0 to 71042
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_contrato   71043 non-null  int64  
 1   inadimplente  71043 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.1 MB


## **Converter Datas**

In [16]:
# listar as colunas que precisam de conversão em cada DataFrame
colunas_data_emprestimos = [
    'data_decisao',
    'data_liberacao',
    'data_primeiro_vencimento',
    'data_ultimo_vencimento_original',
    'data_ultimo_vencimento',
    'data_encerramento'
]

colunas_data_parcelas = [
    'data_prevista_pagamento',
    'data_real_pagamento'
]

colunas_data_cadastral = [
    'data_nascimento'
]

df_parcelas = convert_to_datetime(df_parcelas, colunas_data_parcelas)
df_emprestimos = convert_to_datetime(df_emprestimos, colunas_data_emprestimos)
df_cadastral = convert_to_datetime(df_cadastral, colunas_data_cadastral)


--- Verificando tipos de dados ANTES da conversão ---
data_prevista_pagamento    object
data_real_pagamento        object
dtype: object



--- Verificando tipos de dados DEPOIS da conversão ---
data_prevista_pagamento    datetime64[ns]
data_real_pagamento        datetime64[ns]
dtype: object

--- Verificando tipos de dados ANTES da conversão ---
data_decisao                       object
data_liberacao                     object
data_primeiro_vencimento           object
data_ultimo_vencimento_original    object
data_ultimo_vencimento             object
data_encerramento                  object
dtype: object

--- Verificando tipos de dados DEPOIS da conversão ---
data_decisao                       datetime64[ns]
data_liberacao                     datetime64[ns]
data_primeiro_vencimento           datetime64[ns]
data_ultimo_vencimento_original    datetime64[ns]
data_ultimo_vencimento             datetime64[ns]
data_encerramento                  datetime64[ns]
dtype: object

--- Verificando tipos de dados ANTES da conversão ---
data_nascimento    object
dtype: object

--- Verificando tipos de dados DEPOIS da conversão ---
data

## **Préprocessamento**

Extração de Features
- Features Cadastrais:
  - idade_cliente: Idade do cliente (diferença em anos entre a data atual e a data de nascimento do cliente)
  - renda_por_familiar: Quociente entre a renda anual do cliente e a quantidade de membros da sua família
  - comprometimento_de_renda: Quociente entre o valor do crédito e a renda anual do cliente
- Features históricas:
  - num_emp_aceitos_6m: Número de empréstimos solicitados pelo cliente que foram aceitos nos últimos 6 meses
  - atraso_medio: Média do número de dias atrasados por cliente até o momento

In [17]:
df_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_cliente                  40000 non-null  int64         
 1   sexo                        40000 non-null  object        
 2   data_nascimento             40000 non-null  datetime64[ns]
 3   qtd_filhos                  40000 non-null  int64         
 4   qtd_membros_familia         40000 non-null  float64       
 5   renda_anual                 40000 non-null  float64       
 6   tipo_renda                  40000 non-null  object        
 7   ocupacao                    27324 non-null  object        
 8   tipo_organizacao            40000 non-null  object        
 9   nivel_educacao              40000 non-null  object        
 10  estado_civil                40000 non-null  object        
 11  tipo_moradia                40000 non-null  object    

In [18]:
data_atual_idade = pd.to_datetime('today')
df_cadastral['idade_cliente'] = ((data_atual_idade - df_cadastral['data_nascimento']).dt.days / 365.25).astype(int)

df_cadastral = df_cadastral.drop(columns=['data_nascimento', 'sexo'])

In [19]:
df_cadastral['reda_por_familiar'] = df_cadastral['renda_anual'] / df_cadastral['qtd_membros_familia']

In [20]:
include_columns = [
                  'id_contrato',
                  'id_cliente',
                  'dia_semana_solicitacao',
                  'hora_solicitacao',
                  'tipo_contrato',
                  'valor_credito',
                  'valor_bem',
                  'valor_parcela',
                  'data_decisao'
                  ]

df = df_emprestimos.copy()
df = df[include_columns]

In [21]:
df_merged = df.merge(df_cadastral, on='id_cliente', how='left')
df_merged = df_merged.merge(df_targets, on='id_contrato', how='right')

In [22]:
df_merged['data_decisao'].max()

Timestamp('2024-05-04 00:00:00')

In [23]:
# Informações iniciais
original_count = len(df_merged)

cutoff_str = "01/08/2018"
cutoff_date = pd.to_datetime(cutoff_str, format='%d/%m/%Y')

df_merged = df_merged[df_merged['data_decisao'] >= cutoff_date].copy()

removed_count = original_count - len(df_merged)

if len(df_merged) == 0:
    print(f"Foram removidos {removed_count} registros. O DataFrame resultante está vazio.")
else:
    print(f"Removidos {removed_count} registros.")
    print(f"Novo período: de {df_merged['data_decisao'].min().date()} até {df_merged['data_decisao'].max().date()} ({len(df_merged)} registros)")

Removidos 10182 registros.
Novo período: de 2018-08-01 até 2024-05-04 (60861 registros)


In [24]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60861 entries, 10182 to 71042
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_contrato                 60861 non-null  int64         
 1   id_cliente                  60861 non-null  int64         
 2   dia_semana_solicitacao      60861 non-null  object        
 3   hora_solicitacao            60861 non-null  int64         
 4   tipo_contrato               60861 non-null  object        
 5   valor_credito               60861 non-null  float64       
 6   valor_bem                   58605 non-null  float64       
 7   valor_parcela               60860 non-null  float64       
 8   data_decisao                60861 non-null  datetime64[ns]
 9   qtd_filhos                  60861 non-null  int64         
 10  qtd_membros_familia         60861 non-null  float64       
 11  renda_anual                 60861 non-null  float64    

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from itertools import combinations

class FeatureAutomator:
    """
    Versão que inclui um imputer categórico e o RankCountVectorizer.
    """
    def __init__(self, num_cols, cat_cols, date_col):
        self.num_cols = num_cols
        self.cat_cols = cat_cols
        self.date_col = date_col
        self.poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
        self.num_col_pairs = list(combinations(num_cols, 2))
        self.cat_col_pairs = list(combinations(cat_cols, 2))
        
        # Imputers para numéricos e categóricos
        self.num_imputer = SimpleImputer(strategy='median')
        self.cat_imputer = SimpleImputer(strategy='constant', fill_value='NA') 
        
        # Encoder Categórico
        self.rank_count_vectorizer = RankCountVectorizer() 
        
        self.poly_feature_names_ = None

    def fit(self, df):
        """
        Ajusta todos os transformadores (imputers, encoder, poly) nos dados de treino.
        """
        print("Ajustando (fit) os transformadores...")
        
        self.num_imputer.fit(df[self.num_cols])
        self.cat_imputer.fit(df[self.cat_cols])
        
        # Ajusta o RankCountVectorizer nos dados categóricos já imputados
        df_cat_imputed = pd.DataFrame(self.cat_imputer.transform(df[self.cat_cols]), columns=self.cat_cols, index=df.index)
        self.rank_count_vectorizer.fit(df_cat_imputed, cols=self.cat_cols)
        
        # transforma com o imputer (gera um array NumPy)
        df_num_imputed_np = self.num_imputer.transform(df[self.num_cols])
        # reconstrói o DataFrame para manter os nomes das colunas
        df_num_imputed_df = pd.DataFrame(df_num_imputed_np, columns=self.num_cols, index=df.index)
        # ajusta o PolynomialFeatures no DataFrame com nomes
        self.poly.fit(df_num_imputed_df)
        
        self.poly_feature_names_ = self.poly.get_feature_names_out(self.num_cols)
        self.interaction_feature_names_ = [name for name in self.poly_feature_names_ if ' ' in name]
        
        print("Ajuste concluído.")
        return self

    def transform(self, df):
        """Aplica todas as transformações de features."""
        df_transformed = df.copy()
        print("\nIniciando a transformação (transform)...")

        # ETAPA 0: Imputação de dados numéricos e categóricos
        print("0. Aplicando imputação de valores ausentes...")
        df_transformed[self.num_cols] = self.num_imputer.transform(df_transformed[self.num_cols])
        df_transformed[self.cat_cols] = self.cat_imputer.transform(df_transformed[self.cat_cols]) # NOVO

        # As features de interação e frequência devem ser criadas ANTES do encoding final
        print("1. Criando features de data...")
        df_transformed = self._create_date_features(df_transformed)
        print("2. Criando features de razão numérica...")
        df_transformed = self._create_numerical_ratios(df_transformed)
        print("3. Criando features de interação numérica...")
        df_transformed = self._create_polynomial_features(df_transformed)
        print("4. Aplicando RankCountVectorizer nas colunas categóricas originais...")
        df_transformed = self.rank_count_vectorizer.transform(df_transformed, cols=self.cat_cols) # NOVO

        print("Transformação de features concluída!")
        return df_transformed

    # O resto das funções internas permanecem as mesmas...
    def _create_date_features(self, df):
        if self.date_col in df.columns:
            df[f'{self.date_col}_ano'] = df[self.date_col].dt.year
            df[f'{self.date_col}_mes'] = df[self.date_col].dt.month
            # garantir que a coluna é datetime antes de extrair componentes
            df[self.date_col] = pd.to_datetime(df[self.date_col], errors='coerce')
            df[f'{self.date_col}_dia_do_mes'] = df[self.date_col].dt.day
            df = df.drop(columns=[self.date_col])
        return df
    
    def _create_numerical_ratios(self, df):
        if 'valor_credito' in df.columns and 'renda_anual' in df.columns:
            df['ratio_credito_renda'] = df['valor_credito'] / (df['renda_anual'] + 1e-6)
        return df

    def _create_polynomial_features(self, df):
        interactions = self.poly.transform(df[self.num_cols])
        df_interactions = pd.DataFrame(interactions, columns=self.poly_feature_names_, index=df.index)
        for col in self.interaction_feature_names_:
            new_col_name = 'inter_' + col.replace(' ', '_x_')
            df[new_col_name] = df_interactions[col]
        return df
    
    import pandas as pd

In [26]:
import pandas as pd
import numpy as np

def create_historical_features(df, num_cols, date_col='data_decisao', time_windows_months=[3, 6], aggs=['mean','std','min','max','sum']):

    print("Iniciando criação de features históricas...")
    
    # 1. Preparar o DataFrame base, garantindo a data e um índice único
    base_df = df[['id_cliente', date_col] + num_cols].copy()
    base_df[date_col] = pd.to_datetime(base_df[date_col])
    # Usar o índice original como um identificador único para cada empréstimo
    base_df = base_df.reset_index().rename(columns={'index': 'id_original_emprestimo'})

    # 2. O Self-Join: Juntar o DataFrame com ele mesmo no 'id_cliente'
    # 'df_atual' representa cada empréstimo individualmente (lado esquerdo)
    # 'df_hist' representa todo o histórico disponível para aquele contrato (lado direito)
    df_merged = pd.merge(
        base_df,
        base_df,
        on='id_cliente',
        suffixes=('_atual', '_hist')
    )

    # 3. Filtro Temporal: Manter apenas registros onde o histórico é ANTERIOR ao atual
    df_merged = df_merged[df_merged[f'{date_col}_hist'] < df_merged[f'{date_col}_atual']].copy()

    # 4. Calcular a diferença em dias entre o empréstimo atual e cada um do seu histórico
    df_merged['time_diff_days'] = (df_merged[f'{date_col}_atual'] - df_merged[f'{date_col}_hist']).dt.days

    df_final = df.copy()

    # 5. Loop para cada janela de tempo para agregar e juntar os resultados
    for w in time_windows_months:
        window_days = int(w * 30)
        print(f"Calculando para a janela de {w} meses ({window_days} dias)...")
        
        # Filtra o histórico para a janela de tempo específica
        df_window = df_merged[df_merged['time_diff_days'] <= window_days]

        # Mapeia os nomes das colunas de histórico (ex: 'valor_credito_hist')
        hist_cols = [f'{col}_hist' for col in num_cols]
        
        # Agrupa pelo ID do empréstimo atual e calcula as estatísticas sobre o histórico
        agg_result = df_window.groupby('id_original_emprestimo_atual')[hist_cols].agg(aggs)
        
        # Achata os nomes das colunas do MultiIndex (ex: ('valor_credito_hist', 'mean'))
        agg_result.columns = [f"hist_{col.replace('_hist', '')}_{w}m_{agg}" for col, agg in agg_result.columns]
        
        # Junta os resultados de volta ao DataFrame final
        df_final = df_final.join(agg_result)

    # Preenche NaNs que surgem (ex: primeiro empréstimo de um cliente não tem histórico)
    hist_feature_cols = [c for c in df_final.columns if isinstance(c, str) and c.startswith('hist_')]
    df_final[hist_feature_cols] = df_final[hist_feature_cols].fillna(0)
    
    print(f"\nCriadas {len(hist_feature_cols)} features históricas para as janelas {time_windows_months} meses.")
    
    return df_final

In [27]:
def create_special_features(df, df_emprestimos, df_parcelas, time_windows_months=[3, 6]):
    """
    Versão vetorizada e de alta performance para criar features de janela temporal fixa.
    - Utiliza self-joins temporais para evitar o uso de .iterrows().
    """
    df_featured = df.copy()
    df_featured['data_decisao'] = pd.to_datetime(df_featured['data_decisao'])
    
    time_windows_days = [w * 30 for w in time_windows_months]

    # --- 1. Cálculo de Empréstimos Aceitos (Vetorizado) ---
    print("Calculando o número de empréstimos aceitos...")
    
    # Prepara o histórico de empréstimos
    hist_emprestimos = df_emprestimos[df_emprestimos['status_contrato'].isin(['Approved', 'Aprovado'])][['id_cliente', 'data_decisao']].copy()
    hist_emprestimos.rename(columns={'data_decisao': 'data_decisao_hist'}, inplace=True)
    hist_emprestimos['data_decisao_hist'] = pd.to_datetime(hist_emprestimos['data_decisao_hist'])

    # Self-join temporal
    merged_emprestimos = pd.merge(df_featured[['id_cliente', 'data_decisao']], hist_emprestimos, on='id_cliente')
    merged_emprestimos = merged_emprestimos[merged_emprestimos['data_decisao_hist'] < merged_emprestimos['data_decisao']]
    merged_emprestimos['time_diff'] = (merged_emprestimos['data_decisao'] - merged_emprestimos['data_decisao_hist']).dt.days

    # Cria colunas de contagem para cada janela
    for window_days in time_windows_days:
        window_months = int(window_days / 30)
        col_name = f'num_emp_aceitos_{window_months}m'
        # Conta condicionalmente usando a soma de uma máscara booleana
        counts = merged_emprestimos[merged_emprestimos['time_diff'] <= window_days].groupby('data_decisao').size()
        df_featured[col_name] = df_featured['data_decisao'].map(counts).fillna(0).astype(int)

    # --- 2. Cálculo de Atraso Médio (Vetorizado) ---
    print("Calculando o atraso médio...")

    # Prepara o histórico de parcelas
    hist_parcelas = df_parcelas[['id_contrato', 'data_prevista_pagamento', 'data_real_pagamento']].copy()
    hist_parcelas['data_prevista_pagamento'] = pd.to_datetime(hist_parcelas['data_prevista_pagamento'])
    hist_parcelas['data_real_pagamento'] = pd.to_datetime(hist_parcelas['data_real_pagamento'])
    hist_parcelas['atraso'] = (hist_parcelas['data_real_pagamento'] - hist_parcelas['data_prevista_pagamento']).dt.days.clip(lower=0)
    hist_parcelas = pd.merge(hist_parcelas, df_emprestimos[['id_contrato', 'id_cliente']], on='id_contrato', how='left')
    hist_parcelas.rename(columns={'data_prevista_pagamento': 'data_prevista_hist'}, inplace=True)

    # Self-join temporal com as parcelas
    merged_parcelas = pd.merge(df_featured[['id_cliente', 'data_decisao']], hist_parcelas, on='id_cliente')
    merged_parcelas = merged_parcelas[merged_parcelas['data_prevista_hist'] < merged_parcelas['data_decisao']]
    merged_parcelas['time_diff'] = (merged_parcelas['data_decisao'] - merged_parcelas['data_prevista_hist']).dt.days
    
    # Calcula a média condicional para cada janela
    for window_days in time_windows_days:
        window_months = int(window_days / 30)
        col_name = f'atraso_medio_{window_months}m'
        
        # Usa np.where para aplicar a condição da janela
        merged_parcelas['atraso_na_janela'] = np.where(merged_parcelas['time_diff'] <= window_days, merged_parcelas['atraso'], np.nan)
        
        # Calcula a média, que ignora NaNs
        avg_delay = merged_parcelas.groupby('data_decisao')['atraso_na_janela'].mean()
        df_featured[col_name] = df_featured['data_decisao'].map(avg_delay).fillna(0)

    return df_featured

In [28]:
numerical_features = ['valor_credito', 'valor_bem', 'valor_parcela']

categorical_features = df_merged.select_dtypes(include=['object']).columns.tolist()
date_feature = 'data_decisao'

In [29]:
df_featured = create_special_features(df_merged, df_emprestimos, df_parcelas)

Calculando o número de empréstimos aceitos...
Calculando o atraso médio...


In [30]:
df_featured = create_historical_features(df_featured, num_cols=numerical_features)

Iniciando criação de features históricas...
Calculando para a janela de 3 meses (90 dias)...
Calculando para a janela de 6 meses (180 dias)...

Criadas 30 features históricas para as janelas [3, 6] meses.


In [31]:
df_featured = df_featured.drop(columns=['id_contrato', 'id_cliente'])

train_end = int(len(df_featured) * 0.70)
validation_end = int(len(df_featured) * 0.85)

# fazer a divisão
df_train = df_featured.iloc[:train_end]
df_val = df_featured.iloc[train_end:validation_end]
df_test = df_featured.iloc[validation_end:]

print(f"Período de Treino: de {df_train['data_decisao'].min().date()} até {df_train['data_decisao'].max().date()} ({len(df_train)} amostras)")
print(f"Período de Validação: de {df_val['data_decisao'].min().date()} até {df_val['data_decisao'].max().date()} ({len(df_val)} amostras)")
print(f"Período de Teste: de {df_test['data_decisao'].min().date()} até {df_test['data_decisao'].max().date()} ({len(df_test)} amostras)")

Período de Treino: de 2018-08-01 até 2023-01-20 (42602 amostras)
Período de Validação: de 2023-01-20 até 2023-07-12 (9129 amostras)
Período de Teste: de 2023-07-12 até 2024-05-04 (9130 amostras)


In [32]:
feature_factory = FeatureAutomator(
    num_cols=numerical_features,
    cat_cols=categorical_features,
    date_col=date_feature
)

# AJUSTE (FIT) a fábrica de features APENAS com os dados de TREINO
feature_factory.fit(df_train)

# TRANSFORME ambos os conjuntos, treino e teste
df_train = feature_factory.transform(df_train)
df_test = feature_factory.transform(df_test)
df_val = feature_factory.transform(df_val)

Ajustando (fit) os transformadores...
Ajuste concluído.



Iniciando a transformação (transform)...
0. Aplicando imputação de valores ausentes...
1. Criando features de data...
2. Criando features de razão numérica...
3. Criando features de interação numérica...
4. Aplicando RankCountVectorizer nas colunas categóricas originais...
Transformação de features concluída!

Iniciando a transformação (transform)...
0. Aplicando imputação de valores ausentes...
1. Criando features de data...
2. Criando features de razão numérica...
3. Criando features de interação numérica...
4. Aplicando RankCountVectorizer nas colunas categóricas originais...
Transformação de features concluída!

Iniciando a transformação (transform)...
0. Aplicando imputação de valores ausentes...
1. Criando features de data...
2. Criando features de razão numérica...
3. Criando features de interação numérica...
4. Aplicando RankCountVectorizer nas colunas categóricas originais...
Transformação de features concluída!


In [33]:
cat_cols = df_train.select_dtypes(include=['object']).info()

if cat_cols is None:
    print("Não há colunas categóricas.")
else:
    print("Colunas categóricas presentes: ", cat_cols)

<class 'pandas.core.frame.DataFrame'>
Index: 42602 entries, 10182 to 52783
Empty DataFrame
Não há colunas categóricas.


In [34]:
count_train_na = df_train.isna().sum().sum()
count_test_na = df_test.isna().sum().sum()

print("Total de valores ausentes no conjunto de treino:", count_train_na)
print("Total de valores ausentes no conjunto de teste:", count_test_na)

Total de valores ausentes no conjunto de treino: 0
Total de valores ausentes no conjunto de teste: 0


In [35]:
print("\nDimensões do X_train com novas features:", df_train.shape)
print("Dimensões do X_test com novas features:", df_test.shape)


Dimensões do X_train com novas features: (42602, 63)
Dimensões do X_test com novas features: (9130, 63)


In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_train.dtypes)


dia_semana_solicitacao                   int64
hora_solicitacao                         int64
tipo_contrato                            int64
valor_credito                          float64
valor_bem                              float64
valor_parcela                          float64
qtd_filhos                               int64
qtd_membros_familia                    float64
renda_anual                            float64
tipo_renda                               int64
ocupacao                                 int64
tipo_organizacao                         int64
nivel_educacao                           int64
estado_civil                             int64
tipo_moradia                             int64
possui_carro                             int64
possui_imovel                            int64
nota_regiao_cliente                      int64
nota_regiao_cliente_cidade               int64
idade_cliente                            int64
reda_por_familiar                      float64
inadimplente 

In [37]:
df_train.to_parquet("../data/train.parquet", index=False)
df_test.to_parquet("../data/test.parquet", index=False)
df_val.to_parquet("../data/validation.parquet", index=False)